In [2]:
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox, BRepPrimAPI_MakeSphere
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Fuse, BRepAlgoAPI_Cut
from OCC.Core.gp import gp_Pnt
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer
import copy

print("🎯 方案一：命令模式 + 形状快照")
print("=" * 40)

class SimpleUndoManager:
    """简化的撤销管理器"""
    
    def __init__(self):
        self.shapes_history = []  # 保存每步的形状列表
        self.current_index = -1   # 当前位置
        self.shapes = []          # 当前形状列表
        
        # 保存初始状态（空列表）
        self.save_state()
        print("📚 撤销管理器初始化完成")
    
    def save_state(self):
        """保存当前状态"""
        # 如果不在历史末尾，清除后续历史
        if self.current_index < len(self.shapes_history) - 1:
            self.shapes_history = self.shapes_history[:self.current_index + 1]
        
        # 深拷贝当前状态
        state_copy = copy.deepcopy(self.shapes)
        self.shapes_history.append(state_copy)
        self.current_index += 1
        
        print(f"💾 保存状态: {len(self.shapes)} 个形状")
    
    def add_shape(self, shape, name="Shape"):
        """添加形状"""
        shape_info = {'shape': shape, 'name': name}
        self.shapes.append(shape_info)
        self.save_state()
        print(f"➕ 添加形状: {name}")
        return len(self.shapes) - 1
    
    def replace_shapes(self, new_shapes, operation_name="Operation"):
        """替换所有形状（用于布尔运算）"""
        self.shapes = new_shapes
        self.save_state()
        print(f"🔄 执行操作: {operation_name}")
    
    def undo(self):
        """撤销"""
        if self.current_index <= 0:
            print("❌ 无法撤销：已在起点")
            return False
        
        self.current_index -= 1
        self.shapes = copy.deepcopy(self.shapes_history[self.current_index])
        print(f"↶ 撤销成功，当前: {len(self.shapes)} 个形状")
        return True
    
    def redo(self):
        """重做"""
        if self.current_index >= len(self.shapes_history) - 1:
            print("❌ 无法重做：已在终点")
            return False
        
        self.current_index += 1
        self.shapes = copy.deepcopy(self.shapes_history[self.current_index])
        print(f"↷ 重做成功，当前: {len(self.shapes)} 个形状")
        return True
    
    def get_shapes(self):
        """获取当前所有形状"""
        return [item['shape'] for item in self.shapes]
    
    def get_status(self):
        """获取状态信息"""
        return {
            'shape_count': len(self.shapes),
            'history_position': f"{self.current_index + 1}/{len(self.shapes_history)}",
            'can_undo': self.current_index > 0,
            'can_redo': self.current_index < len(self.shapes_history) - 1
        }

print("✅ SimpleUndoManager 定义完成")

🎯 方案一：命令模式 + 形状快照
✅ SimpleUndoManager 定义完成


In [4]:
class SimpleModeler:
    """简化的建模器"""
    
    def __init__(self):
        self.undo_manager = SimpleUndoManager()
        print("🎨 简化建模器初始化完成")
    
    def create_box(self, x, y, z):
        """创建立方体"""
        box = BRepPrimAPI_MakeBox(x, y, z).Shape()
        self.undo_manager.add_shape(box, f"Box_{x}x{y}x{z}")
        return box
    
    def create_sphere(self, radius, center=(0, 0, 0)):
        """创建球体"""
        center_point = gp_Pnt(*center)
        sphere = BRepPrimAPI_MakeSphere(center_point, radius).Shape()
        self.undo_manager.add_shape(sphere, f"Sphere_R{radius}")
        return sphere
    
    def union(self):
        """对所有形状执行并集"""
        shapes = self.undo_manager.get_shapes()
        if len(shapes) < 2:
            print("❌ 需要至少2个形状才能执行并集")
            return None
        
        result = shapes[0]
        for i in range(1, len(shapes)):
            union_op = BRepAlgoAPI_Fuse(result, shapes[i])
            if union_op.IsDone():
                result = union_op.Shape()
            else:
                print("❌ 并集运算失败")
                return None
        
        # 用结果替换所有形状
        new_shapes = [{'shape': result, 'name': 'UnionResult'}]
        self.undo_manager.replace_shapes(new_shapes, "Union")
        return result
    
    def cut_last_two(self):
        """对最后两个形状执行差集（第一个减去第二个）"""
        shapes = self.undo_manager.get_shapes()
        if len(shapes) < 2:
            print("❌ 需要至少2个形状才能执行差集")
            return None
        
        # 取最后两个形状
        shape1 = shapes[-2]
        shape2 = shapes[-1]
        
        cut_op = BRepAlgoAPI_Cut(shape1, shape2)
        if cut_op.IsDone():
            result = cut_op.Shape()
            
            # 用结果替换最后两个形状
            new_shapes = self.undo_manager.shapes[:-2] + [{'shape': result, 'name': 'CutResult'}]
            self.undo_manager.replace_shapes(new_shapes, "Cut")
            return result
        else:
            print("❌ 差集运算失败")
            return None
    
    def undo(self):
        """撤销"""
        return self.undo_manager.undo()
    
    def redo(self):
        """重做"""
        return self.undo_manager.redo()
    
    def display(self):
        """显示当前所有形状"""
        renderer = JupyterRenderer()
        shapes = self.undo_manager.get_shapes()
        colors = ["#FF6B6B", "#4ECDC4", "#45B7D1", "#96CEB4", "#FECA57"]
        
        for i, shape in enumerate(shapes):
            color = colors[i % len(colors)]
            renderer.DisplayShape(shape, shape_color=color)
        
        print(f"🖼️ 显示 {len(shapes)} 个形状")
        return renderer
    
    def show_status(self):
        """显示状态"""
        status = self.undo_manager.get_status()
        print(f"\n📊 建模器状态:")
        print(f"   形状数量: {status['shape_count']}")
        print(f"   历史位置: {status['history_position']}")
        print(f"   可撤销: {'✅' if status['can_undo'] else '❌'}")
        print(f"   可重做: {'✅' if status['can_redo'] else '❌'}")

print("✅ SimpleModeler 定义完成")

✅ SimpleModeler 定义完成


In [5]:
def demo_simple_undo():
    """演示简化的撤销/重做系统"""
    
    print("\n🎭 方案一演示：命令模式")
    print("=" * 30)
    
    modeler = SimpleModeler()
    
    # 步骤1：创建几何体
    print("\n📦 创建几何体:")
    modeler.create_box(20, 20, 20)
    modeler.create_sphere(15, center=(10, 10, 10))
    modeler.show_status()
    
    # 步骤2：执行并集
    print("\n🔧 执行并集:")
    modeler.union()
    modeler.show_status()
    
    # 步骤3：添加更多形状
    print("\n📦 添加更多形状:")
    modeler.create_box(8, 8, 30)
    modeler.show_status()
    
    # 步骤4：执行差集
    print("\n🔧 执行差集:")
    modeler.cut_last_two()
    modeler.show_status()
    
    # 步骤5：撤销测试
    print("\n↶ 撤销测试:")
    modeler.undo()
    modeler.show_status()
    
    modeler.undo()
    modeler.show_status()
    
    # 步骤6：重做测试
    print("\n↷ 重做测试:")
    modeler.redo()
    modeler.show_status()
    
    # 最终显示
    print("\n🖼️ 最终显示:")
    renderer = modeler.display()
    
    return modeler, renderer

# 运行演示
modeler1, renderer1 = demo_simple_undo()
renderer1.Display()


🎭 方案一演示：命令模式
💾 保存状态: 0 个形状
📚 撤销管理器初始化完成
🎨 简化建模器初始化完成

📦 创建几何体:
💾 保存状态: 1 个形状
➕ 添加形状: Box_20x20x20
💾 保存状态: 2 个形状
➕ 添加形状: Sphere_R15

📊 建模器状态:
   形状数量: 2
   历史位置: 3/3
   可撤销: ✅
   可重做: ❌

🔧 执行并集:
💾 保存状态: 1 个形状
🔄 执行操作: Union

📊 建模器状态:
   形状数量: 1
   历史位置: 4/4
   可撤销: ✅
   可重做: ❌

📦 添加更多形状:
💾 保存状态: 2 个形状
➕ 添加形状: Box_8x8x30

📊 建模器状态:
   形状数量: 2
   历史位置: 5/5
   可撤销: ✅
   可重做: ❌

🔧 执行差集:
💾 保存状态: 1 个形状
🔄 执行操作: Cut

📊 建模器状态:
   形状数量: 1
   历史位置: 6/6
   可撤销: ✅
   可重做: ❌

↶ 撤销测试:
↶ 撤销成功，当前: 2 个形状

📊 建模器状态:
   形状数量: 2
   历史位置: 5/6
   可撤销: ✅
   可重做: ✅
↶ 撤销成功，当前: 1 个形状

📊 建模器状态:
   形状数量: 1
   历史位置: 4/6
   可撤销: ✅
   可重做: ✅

↷ 重做测试:
↷ 重做成功，当前: 2 个形状

📊 建模器状态:
   形状数量: 2
   历史位置: 5/6
   可撤销: ✅
   可重做: ✅

🖼️ 最终显示:
🖼️ 显示 2 个形状
